Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [22]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables
  hidden_layers = 1024
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers]))
  B1 = tf.Variable(tf.zeros([hidden_layers]))
  W2 = tf.Variable(tf.truncated_normal([hidden_layers, num_labels]))
  B2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1), W2) + B2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    0.01 * tf.nn.l2_loss(W1) +
    0.001 * tf.nn.l2_loss(W2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1), W2) + B2)

  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1), W2) + B2)

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3463.479980
Minibatch accuracy: 7.0%
Validation accuracy: 35.6%
Minibatch loss at step 500: 24.209116
Minibatch accuracy: 75.0%
Validation accuracy: 70.1%
Minibatch loss at step 1000: 1.710412
Minibatch accuracy: 78.9%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 1.064210
Minibatch accuracy: 84.4%
Validation accuracy: 84.5%
Minibatch loss at step 2000: 0.932078
Minibatch accuracy: 87.5%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 0.771208
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 0.856001
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Test accuracy: 91.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [24]:
batch_size = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables
  hidden_layers = 1024
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers]))
  B1 = tf.Variable(tf.zeros([hidden_layers]))
  W2 = tf.Variable(tf.truncated_normal([hidden_layers, num_labels]))
  B2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1), W2) + B2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    0.01 * tf.nn.l2_loss(W1) +
    0.001 * tf.nn.l2_loss(W2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1), W2) + B2)

  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1), W2) + B2)

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3534.939453
Minibatch accuracy: 10.0%
Validation accuracy: 18.1%
Minibatch loss at step 500: 6444.170898
Minibatch accuracy: 40.0%
Validation accuracy: 49.7%
Minibatch loss at step 1000: 139.800339
Minibatch accuracy: 40.0%
Validation accuracy: 44.7%
Minibatch loss at step 1500: 23.501373
Minibatch accuracy: 30.0%
Validation accuracy: 21.0%
Minibatch loss at step 2000: 14.535149
Minibatch accuracy: 20.0%
Validation accuracy: 22.5%
Minibatch loss at step 2500: 9.677967
Minibatch accuracy: 30.0%
Validation accuracy: 22.2%
Minibatch loss at step 3000: 6.925723
Minibatch accuracy: 0.0%
Validation accuracy: 23.1%
Test accuracy: 24.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [52]:
batch_size = 20

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  keep_prob = tf.placeholder("float")
    
  # Variables
  hidden_layers = 1024
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers]))
  B1 = tf.Variable(tf.zeros([hidden_layers]))
  W2 = tf.Variable(tf.truncated_normal([hidden_layers, num_labels]))
  B2 = tf.Variable(tf.zeros([num_labels]))
    
  hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1), keep_prob)
  
  # Training computation.
  logits = tf.matmul(hidden_layer, W2) + B2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    0.01 * tf.nn.l2_loss(W1) +
    0.001 * tf.nn.l2_loss(W2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1), W2) + B2)

  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1), W2) + B2)

In [53]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3685.701904
Minibatch accuracy: 15.0%
Validation accuracy: 24.4%
Minibatch loss at step 500: 131159539712.000000
Minibatch accuracy: 30.0%
Validation accuracy: 38.7%
Minibatch loss at step 1000: 521459332391370752.000000
Minibatch accuracy: 15.0%
Validation accuracy: 35.6%
Minibatch loss at step 1500: 851812851133488675422208.000000
Minibatch accuracy: 50.0%
Validation accuracy: 48.3%
Minibatch loss at step 2000: 8520056433924460744703200985088.000000
Minibatch accuracy: 45.0%
Validation accuracy: 51.6%
Minibatch loss at step 2500: nan
Minibatch accuracy: 10.0%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [74]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables
  hidden_layers = 1024
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers]))
  B1 = tf.Variable(tf.zeros([hidden_layers]))
  W2 = tf.Variable(tf.truncated_normal([hidden_layers, num_labels]))
  B2 = tf.Variable(tf.zeros([num_labels]))
    
  hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + B1)
  
  # Training computation.
  logits = tf.matmul(hidden_layer, W2) + B2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    0.01 * tf.nn.l2_loss(W1) +
    0.001 * tf.nn.l2_loss(W2))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 7500, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + B1), W2) + B2)

  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + B1), W2) + B2)

In [75]:
num_steps = 7501

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3473.409668
Minibatch accuracy: 9.4%
Validation accuracy: 26.7%
Minibatch loss at step 500: 1157.448608
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 426.829956
Minibatch accuracy: 79.7%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 158.702301
Minibatch accuracy: 85.9%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 60.129757
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 23.520977
Minibatch accuracy: 86.7%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 9.968788
Minibatch accuracy: 82.0%
Validation accuracy: 86.2%
Minibatch loss at step 3500: 4.516671
Minibatch accuracy: 91.4%
Validation accuracy: 86.9%
Minibatch loss at step 4000: 2.546705
Minibatch accuracy: 89.8%
Validation accuracy: 87.7%
Minibatch loss at step 4500: 1.842748
Minibatch accuracy: 90.6%
Validation accuracy: 87.6%
Minibatch loss at step 5000: 1.555822
Minibatch accuracy: 85.2%
Validat